In [1]:
import os
import re
import csv
import sys
import math
import heapq
import shutil
import queue as Q
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba
import nltk
from model import TrieNode

In [2]:
folder = os.path.exists('blocks')
if not folder:
    os.makedirs('blocks')

In [3]:
ori_path = '动漫'
files= os.listdir(ori_path)

In [4]:
def construct(intToJuji, intToFanju):
    i = 0
    j = -1000
    k = 0
    terms_id = dict()
    for root, dirs, files in os.walk(ori_path, topdown=True):
        if '\.ipynb_checkpoints' in root:
            continue
        if len(files) == 0:
            j+=1000
            i=0
            intToFanju[j] = root
        else:
            i+=1
            key = j+i
            intToJuji[key] = root
            for name in files:
                f = open(root+"\\"+ name, errors='ignore')
                lines = f.readlines()
                texts_s,texts_l = arrayTodict(lines)
                counts = cut3(texts_s)
            for word in counts:
                terms_id[word] = terms_id.get(word, 0) + counts[word]
            #if (sys.getsizeof(terms_id) > 10000000):
             #   for key_word in terms_id:
              #      jieba.add_word(key_word, freq = 100*terms_id[key_word], tag = None)
               # terms_id = dict()
    return terms_id

In [ ]:
def construct(intToJuji, intToFanju):
    i = 0
    j = -1000
    k = 0
    terms_id = dict()
    for root, dirs, files in os.walk(ori_path, topdown=True):
        if '\.ipynb_checkpoints' in root:
            continue
        if len(files) == 0:
            j+=1000
            i=0
            intToFanju[j] = root
        else:
            i+=1
            key = j+i
            intToJuji[key] = root
            for name in files:
                f = open(root+"\\"+ name, errors='ignore')
                lines = f.readlines()
                texts_s,texts_l = arrayTodict(lines)
                counts = cut3(texts_s)
            for word in counts:
                terms_id[word] = terms_id.get(word, 0) + counts[word]
            if (sys.getsizeof(terms_id) > 10000000):
                for key_word in terms_id:
                    jieba.add_word(key_word, freq = 100*terms_id[key_word], tag = None)
                    print(key_word,100*terms_id[key_word] )
                terms_id = dict()
    for root, dirs, files in os.walk(ori_path, topdown=True):
        if '\.ipynb_checkpoints' in root:
            continue
        if len(files) == 0:
            j+=1000
            i=0
            intToFanju[j] = root
        else:
            i+=1
            key = j+i
            intToJuji[key] = root
            for name in files:
                f = open(root+"\\"+ name, errors='ignore')
                lines = f.readlines()
                texts_s,texts_l = arrayTodict(lines)
                counts = cut3(texts_s)
                for word in counts:
                    if word in terms_id:
                        terms_id[word].append((key,counts[word]))
                    else: 
                        terms_id[word] = [(key,counts[word])]
                if (sys.getsizeof(terms_id) > 1000000):
                    k += 1
                    build_c(k, terms_id)
                    terms_id = dict()
    build_c(k+1, terms_id)
    terms_id = dict()

In [6]:
intToJuji = dict()
intToFanju= dict()

In [13]:
small_words = construct(intToJuji, intToFanju)

In [14]:
small_words = sorted(small_words.items(),key = lambda item: -item[1])

In [15]:
len(small_words)

2031105

In [16]:
new_words = []
for word in small_words:
    if word[1]!=1:
        new_words.append(word)

In [17]:
sys.getsizeof(small_words)

18280056

In [142]:
s = 'sjkk啥呀'

In [18]:
len(new_words)

586155

In [26]:
path = "new_words_all.txt"
with open(path, 'w', newline='') as f:
    for key in new_words:
        f.write(''+key[0])
        f.write(',')
        f.write(str(key[1]))
        f.write('\n')
    f.close()

In [144]:
for i in superfluous:
    s = s.rstrip(i)

'sjkk'

In [8]:
def shorten(s):
    superfluous = ' 啊吧哇呀嘛吗耶？！?!.。诶了阿哈呗啥'
    for i in superfluous:
        s = s.rstrip(i)
    ss= ''.join(reversed(s))
    k = -1
    j = 0
    array = [-1]
    plen = len(ss)
    zeros = 0
    nonze = 0
    while j < plen:
        if k == -1 or ss[j] == ss[k]:
            k += 1
            j += 1
            if k == 0:
                if j != zeros + 1:
                    loop = (int)(nonze/zeros) -1
                    return cut_string(s, plen) if loop<1 else cut_string(s, plen - zeros*loop)
                zeros += 1
            else:
                nonze += 1
            array.append(k)
        else:
            k = array[k]
    return cut_string(s, plen) if zeros==plen else (cut_string(s, zeros) if zeros>1 else cut_string(s, zeros+1)) 
def cut_string(s, lens):
    return s[0: lens]

In [9]:
def arrayTodict(lines):
    word_dict_s={}
    word_dict_l={}
    for line in lines:
        line=shorten(line.strip('\n').replace(' ', ''))
        if len(line)<2:
            continue
        if len(line)<5:
            if word_dict_s.get(line):
                word_dict_s[line] += 1
            else:
                word_dict_s[line] = 1
            continue
        if word_dict_l.get(line):
            word_dict_l[line] += 1
        else:
            word_dict_l[line] = 1
    return word_dict_s, word_dict_l

In [10]:
def cut1(word_dict):
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        words = jieba.lcut(line)
        for word in words:
            if len(word) == 1:
                continue
            else:
                counts[word] = counts.get(word, 0) + num
    return counts

In [11]:
def cut3(word_dict):
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        counts[line] = counts.get(line, 0) + num
    return counts

In [12]:
def cut2(word_dict):
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        if len(line)<5:
            counts[line] = counts.get(line, 0) + num
            continue
        words = jieba.lcut(line)
        for word in words:
            if len(word) == 1:
                continue
            else:
                counts[word] = counts.get(word, 0) + num
    return counts

In [88]:
def build_c(i, terms_id):
    list1 = []
    path = "chunks/chunk_"+str(i)+".txt"
    list1 = sorted(terms_id.items(), key=lambda item: item[0])
    with open(path, 'w', newline='') as f:
        for key in list1:
            list2= []
            f.write(str(key[0]))
            for word in key[1]:
                f.write(',')
                f.write(str(word[0]))
                f.write('-')
                f.write(str(word[1]))
            f.write('\n')
    f.close()

In [86]:
def make_dirs():
    # Create a new folder to generate inverted indexes
    folder1 = os.path.exists('chunks')
    folder2 = os.path.exists('blocks')
    if not folder1:
        os.makedirs('chunks')
    if not folder2:
        os.makedirs('blocks')

In [87]:
make_dirs()

In [94]:
shorten('2333')

'233'

In [95]:
shorten('23333333')

'233'

In [96]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}

    def inc(self, numOccur):
        self.count += numOccur

    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)
    def disp_plus(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            if child.count > 10:
                child.disp_plus(ind+1)

In [97]:
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
def updateFPtree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 判断items的第一个结点是否已作为子结点
        inTree.children[items[0]].inc(count)
    else:
        # 创建新的分支
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新相应频繁项集的链表，往后添加
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    # 递归
    if len(items) > 1:
        updateFPtree(items[1::], inTree.children[items[0]], headerTable, count)

def createFPtree(word_dict, cutted, minSup=1):
    headerTable = {}
    for trans in word_dict:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    freqItemSet = set(headerTable.keys()) # 满足最小支持度的频繁项集
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] # element: [count, node]

    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        # dataSet：[element, count]
        localD = {}
        for item in tranSet:
            if item in freqItemSet: # 过滤，只取该样本中满足最小支持度的频繁项
                localD[item] = headerTable[item][0] # element : count
        if len(localD) > 0:
            # 根据全局频数从大到小对单样本排序
            orderedItem = [v[0] for v in localD.items()]
            # 用过滤且排序后的样本更新树
            updateFPtree(orderedItem, retTree, headerTable, count)
    return retTree, headerTable

In [101]:
print ("/".join(jieba.cut("大乔为你点赞")))

大乔/为/你/点赞


In [110]:
jieba.add_word("大乔", freq = 10000, tag = None)
print ("/".join(jieba.cut("大乔为你点赞")))

大乔/为你点赞
